In [11]:
# !/use/bin/env python
# encoding:utf-8
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.model_selection import train_test_split
import math
import easy_excel
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import *
import sklearn.ensemble
import joblib
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV
import subprocess
from sklearn.utils import shuffle
import itertools
name="KNN"

In [2]:
def read_fasta_file(path):
    '''
    used for load fasta data and transformd into numpy.array format
    '''
    fh=open(m6a_benchmark_dataset)
    seq=[]
    for line in fh:
        if line.startswith('>'):
            continue
        else:
            seq.append(line.replace('\r','').replace('\n',''))
    fh.close()
    matrix_data=np.array([list(e) for e in seq])
    return matrix_data

In [3]:
def Comparing_score(query_sequence,original_sequence):
    
    score=0
    for index,data in enumerate(query_sequence):
        if data==original_sequence[index]:
            score=score+2
        else:
            score=score-1
    return score


In [4]:
def generating_one_column(matrix_data):
    whole_comparison_score=[]
    the_begin_of_index=len(matrix_data)/2
    for index_1,data_1 in enumerate(matrix_data):
        one_line_comparison_score=np.zeros(len(matrix_data))
        mark_origin_label=np.zeros(len(matrix_data))
        for index_2,data_2 in enumerate(matrix_data):
            if index_1!=index_2:
                one_line_comparison_score[index_1]=-100
                if index_2<the_begin_of_index:
                    mark_origin_label[index_2]=1
                one_line_comparison_score[index_2]=Comparing_score(data_1,data_2)
        temp=[]
        temp=[one_line_comparison_score,mark_origin_label]
        whole_comparison_score.append(temp)
    return whole_comparison_score

In [5]:
def generating_features(matrix_data,K_list):
    matrix=generating_one_column(matrix_data)
    whole_=[]
    for index,K_data in enumerate(K_list):
        line=[]
        for data in matrix:
            idx=np.argsort(data[0])[::-1]
            idx=idx[xrange(K_data)]
            data[1]=pd.DataFrame(data[1])
            datas=data[1].iloc[idx]
            datas=datas.values
            line.append(sum(datas)/K_data)
        whole_.append(line)
    whole_=np.array(whole_).T
    return whole_
            
            
    

In [6]:

m6a_benchmark_dataset='/home02/chenhuangrong/december_data/m6a_data.txt'
matrix_data=read_fasta_file(m6a_benchmark_dataset)
final_feature_matrix=generating_features(matrix_data,[10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200])
pd.DataFrame(final_feature_matrix[0]).to_csv('/home02/chenhuangrong/december_data/'+name+'.csv',header=None,index=False)




In [12]:
def make_kmer_list(k, alphabet):
    try:
        return ["".join(e) for e in itertools.product(alphabet, repeat=k)]
    except TypeError:
        print("TypeError: k must be an inter and larger than 0, alphabet must be a string.")
        raise TypeError
    except ValueError:
        print("TypeError: k must be an inter and larger than 0")
        raise ValueError

In [13]:
def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall=0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN

In [70]:
"""
used to merge features
"""
RFH_=pd.read_csv("/home02/chenhuangrong/PSNP.csv",header=None,index_col=None)#sys.argv[1])
RFH_=RFH_.values
RFH_=pd.DataFrame(RFH_).astype(float)
PseDNC_=pd.read_csv("/home02/chenhuangrong/PSDNP_1_gap.csv",header=None,index_col=None)
PseDNC_=pd.DataFrame(PseDNC_).astype(float)
print "first_file_num:",len(RFH_)
print "first_file_length:",len(RFH_.values[0])
print "second_file_num:",len(PseDNC_)
print "second_file_length:",len(PseDNC_.values[0])
RFH_PseDNC=pd.concat([RFH_,PseDNC_],axis=1)
print "output_file_num:",len(RFH_PseDNC)
print "output_file_len:",len(RFH_PseDNC.values[0])
scaler=MinMaxScaler()
RFH_PseDNC=scaler.fit_transform(np.array(RFH_PseDNC))

first_file_num: 2614
first_file_length: 3
second_file_num: 2614
second_file_length: 3
output_file_num: 2614
output_file_len: 6


In [20]:
"""
    cross validation
"""

datapath ='/home02/chenhuangrong/december_data/'+name+'.csv'
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) / 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) / 2)))
Y.extend(Y2)
Y = np.array(Y)
svc = svm.SVC()
parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,7,9)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,7))}
clf = GridSearchCV(svc, parameters, cv=10, n_jobs=-1, scoring='accuracy')
clf.fit(X, Y)
C=clf.best_params_['C']
# joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
# print clf.best_score_
gamma=clf.best_params_['gamma']
# C=128
# gamma=0.03125
y_predict=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=12)
ROC_AUC_area="%0.4f"%cross_val_score(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=12).mean()
y_predict_prob=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,n_jobs=12,method='predict_proba')
predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
predict_save=np.array(predict_save).T
pd.DataFrame(predict_save).to_csv('/home02/chenhuangrong/december_data/KNN_predict.csv',header=None,index=False)
ACC=metrics.accuracy_score(Y,y_predict)
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
F1_Score=metrics.f1_score(Y, y_predict)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(Y, y_predict)
pos=TP+FN
neg=FP+TN
savedata=[[['svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
print savedata
print X.shape[1]
easy_excel.save("svm_crossvalidation",[str(X.shape[1])],savedata,'/home02/chenhuangrong/december_data/cross_validation_'+name+'.xls')
# best_ACC is 0.7157 with C=[-2,7]
# y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
# Y=pd.DataFrame(Y)    
# y_predict_proba=pd.DataFrame(y_predict_proba)
# y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
# pd.DataFrame(y_predict_proba).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_label.csv',header=None,index=False)
# y_predict=pd.DataFrame(y_predict)
# y_predict_=pd.concat([Y,y_predict],axis=1)
# pd.DataFrame(y_predict_).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_predict.csv',header=None,index=False)

[[['svmC:128.0gamma:0.03125', 0.71576128538638106, 0.7270531400966184, 0.6908951798010712, 0.6908951798010712, 0.740627390971691, 0.7153292210241272, 0.70851314240878771, 0.70851314240878771, 0.43205720354465693, '0.7157', 903.0, 404.0, 339.0, 968.0, 1307.0, 1307.0]]]
20


True